In [2]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

# Simple Map showing the Testing Locations

In [2]:
m = folium.Map(
    location=[41.883844, -87.632162],
    tiles='openstreetmap',
    zoom_start=10,
    max_zoom = 10,
    min_zoom = 10
    
)



#folium.tooltip = 'Click Me!' #WARNING: Tooltips are not in this release; use the dev one if you want it



folium.Marker([41.973312, -87.910576], popup='OHare Iternational Airport', icon=folium.Icon(icon='plane', color = 'red')).add_to(m)
folium.Marker([41.786, -87.752], popup='Midway International Airport', icon=folium.Icon(icon='plane', color = 'green')).add_to(m)

m

# Weather EDA Starts HERE

In [3]:
weather_raw = pd.read_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Proj4WeatherWithLoc.csv')

In [4]:
weather_raw.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure (in hg),SeaLevel (in hg),ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0,0,29.1,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0,0,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0,0,29.39,30.12,11.7,7,11.9


In [5]:
weather_raw['Station'] = weather_raw['Station'].astype(str)

#Converting to string so I can run the Map function below.

In [6]:
weather_raw["Station Location"] = weather_raw["Station"].map(lambda x: "O'HARE" if '1' in x else "MIDWAY" if '2' in x else "")


In [7]:
weather_raw['Station'] = weather_raw['Station'].astype(int)
#And back to integer now for math usage

In [8]:
# O'Hare: 41.970748, -87.908336
# Midway: 41.787957, -87.752359

weather_raw["Latitude"] = weather_raw["Station Location"].map(lambda x: "41.970748" if "O'HARE" in x else "41.787957" if "MIDWAY" in x else "")
weather_raw["Longitude"] = weather_raw["Station Location"].map(lambda x: "-87.908336" if "O'HARE" in x else "-87.752359" if "MIDWAY" in x else "")

#Mapping latitude and longitude for the airports

In [9]:
weather_raw["Latitude"] = weather_raw["Latitude"].astype(float)
weather_raw["Longitude"] = weather_raw["Longitude"].astype(float)

#Converting the string entries of location back into floats for math usage

In [10]:
weather_raw['Date'] = pd.to_datetime(weather_raw['Date'], format='%Y/%m/%d')

In [11]:
weather_raw.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure (in hg),SeaLevel (in hg),ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0,0,29.1,29.82,1.7,27,9.2,O'HARE,41.970748,-87.908336
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0,29.18,29.82,2.7,25,9.6,MIDWAY,41.787957,-87.752359
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0,0,29.38,30.09,13.0,4,13.4,O'HARE,41.970748,-87.908336
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,0,29.44,30.08,13.3,2,13.4,MIDWAY,41.787957,-87.752359
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0,0,29.39,30.12,11.7,7,11.9,O'HARE,41.970748,-87.908336


In [12]:
weather_raw.Date.dtype

dtype('<M8[ns]')

In [13]:
weather2011 = weather_raw.loc[(weather_raw['Date'] > '2011-05-01') & (weather_raw['Date'] <= '2011-10-31')]
weather2013 = weather_raw.loc[(weather_raw['Date'] > '2013-05-01') & (weather_raw['Date'] <= '2013-10-31')]

#Making a subset of the weather data over just these years as thats all the spray data we have.

In [14]:
weather2011.reset_index(drop=True)
weather2013.reset_index(drop=True)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure (in hg),SeaLevel (in hg),ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2013-05-02,55,42,49,-5,41,45,16,0,...,0,0.11,29.44,30.17,15.2,3,15.4,O'HARE,41.970748,-87.908336
1,2,2013-05-02,71,42,M,M,39,45,M,M,...,M,0,29.51,30.17,15.8,2,16.1,MIDWAY,41.787957,-87.752359
2,1,2013-05-03,48,40,44,-10,42,43,21,0,...,0,0.05,29.28,30.03,11.2,4,11.5,O'HARE,41.970748,-87.908336
3,2,2013-05-03,50,40,45,M,41,44,20,0,...,M,0.02,29.34,30.03,11.0,2,11.2,MIDWAY,41.787957,-87.752359
4,1,2013-05-04,71,48,60,6,45,51,5,0,...,0,0.02,29.26,29.99,6.1,6,7,O'HARE,41.970748,-87.908336
5,2,2013-05-04,72,49,61,M,44,51,4,0,...,M,T,29.32,29.99,6.0,6,7.6,MIDWAY,41.787957,-87.752359
6,1,2013-05-05,69,48,59,4,43,50,6,0,...,0,T,29.28,30,8.8,3,9.3,O'HARE,41.970748,-87.908336
7,2,2013-05-05,70,49,60,M,41,50,5,0,...,M,0,29.34,30,8.0,2,8.6,MIDWAY,41.787957,-87.752359
8,1,2013-05-06,71,50,61,6,39,49,4,0,...,0,T,29.28,30,7.5,3,8,O'HARE,41.970748,-87.908336
9,2,2013-05-06,70,51,61,M,37,49,4,0,...,M,0,29.34,30,7.0,2,7.5,MIDWAY,41.787957,-87.752359


# Spray EDA Starts Here

In [15]:
spray_raw = pd.read_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/spray.csv')

In [16]:
spray_raw.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [17]:
spray_raw['CombinedLoc'] = spray_raw.Latitude.astype(str).str.cat(spray_raw.Longitude.astype(str), sep=', ')

In [18]:
spray_raw.Date.dtype

dtype('O')

In [19]:
spray_raw['Date'] = pd.to_datetime(spray_raw['Date'], format='%Y/%m/%d')

In [20]:
#2011: 08-29 to 09/07
#2013: 07-17 to 09/05

spray2011 = spray_raw.loc[(spray_raw['Date'] > '2011-08-29') & (spray_raw['Date'] <= '2011-09-07')]
spray2013 = spray_raw.loc[(spray_raw['Date'] > '2013-07-17') & (spray_raw['Date'] <= '2013-09-05')]

In [21]:
spray2011.reset_index(drop=True)

,Date,Time,Latitude,Longitude,CombinedLoc
0,2011-09-07,7:52:52 PM,41.981433,-87.787777,"41.9814333333, -87.7877766667"
1,2011-09-07,7:53:02 PM,41.980998,-87.787778,"41.9809983333, -87.7877783333"
2,2011-09-07,7:53:12 PM,41.980560,-87.787762,"41.98056, -87.7877616667"
3,2011-09-07,7:53:22 PM,41.980198,-87.787758,"41.9801983333, -87.7877583333"
4,2011-09-07,7:53:32 PM,41.979752,-87.787765,"41.9797516667, -87.787765"
5,2011-09-07,7:53:42 PM,41.979352,-87.787753,"41.9793516667, -87.7877533333"
6,2011-09-07,7:53:52 PM,41.978850,-87.787763,"41.97885, -87.7877633333"
7,2011-09-07,7:54:02 PM,41.978398,-87.787773,"41.9783983333, -87.7877733333"
8,2011-09-07,7:54:12 PM,41.977990,-87.787760,"41.97799, -87.78776"
9,2011-09-07,7:54:22 PM,41.977495,-87.787778,"41.977495, -87.7877783333"


In [22]:
spray2011.Date.astype(str)

95      2011-09-07
96      2011-09-07
97      2011-09-07
98      2011-09-07
99      2011-09-07
100     2011-09-07
101     2011-09-07
102     2011-09-07
103     2011-09-07
104     2011-09-07
105     2011-09-07
106     2011-09-07
107     2011-09-07
108     2011-09-07
109     2011-09-07
110     2011-09-07
111     2011-09-07
112     2011-09-07
113     2011-09-07
114     2011-09-07
115     2011-09-07
116     2011-09-07
117     2011-09-07
118     2011-09-07
119     2011-09-07
120     2011-09-07
121     2011-09-07
122     2011-09-07
123     2011-09-07
124     2011-09-07
           ...    
2179    2011-09-07
2180    2011-09-07
2181    2011-09-07
2182    2011-09-07
2183    2011-09-07
2184    2011-09-07
2185    2011-09-07
2186    2011-09-07
2187    2011-09-07
2188    2011-09-07
2189    2011-09-07
2190    2011-09-07
2191    2011-09-07
2192    2011-09-07
2193    2011-09-07
2194    2011-09-07
2195    2011-09-07
2196    2011-09-07
2197    2011-09-07
2198    2011-09-07
2199    2011-09-07
2200    2011

## Spraying locations in 2011

In [58]:
from folium import plugins
from folium.plugins import MarkerCluster

location2011 = spray2011['Latitude'].mean(), spray2011['Longitude'].mean()
# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()

map2011 = (folium.Map(location=location2011,zoom_start=13))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=3,
                        fill = True,
                        color = "#4286f4", 
                        fill_color="#4286f4"
                       ).add_to(map2011)

map2011.save('/Users/skiparay/Desktop/2011 Spray.html') 

## Spraying Locations in 2013

In [61]:
map2013 = (folium.Map(location=[41.977049, -87.768738],zoom_start=11))

for index,row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']],
                       radius=3,
                        fill = True,
                        color = "#b22323", 
                        fill_color="#b22323", # divvy color
                       ).add_to(map2013)

map2013.save('/Users/skiparay/Desktop/2013 Spray.html')

## All Spraying Locations (2011 & 2013), clustered together

In [63]:
masterspraycluster = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

marker_cluster = MarkerCluster().add_to(masterspraycluster)

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(marker_cluster)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(marker_cluster)

masterspraycluster.add_child(folium.LatLngPopup())
masterspraycluster.save('/Users/skiparay/Desktop/Master Spray (with Clusters).html')

## All Spraying Locations (2011 & 2013), Raw

In [64]:
masterspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(masterspray)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(masterspray)

masterspray.add_child(folium.LatLngPopup())
masterspray.save('/Users/skiparay/Desktop/Master Spray.html')

In [54]:
# markerspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()
# locationlist2013 = spray2013[["Latitude","Longitude"]].values.tolist()
# icon2011 = folium.Icon(color='red',icon='ok-sign')
# icon2013 = folium.Icon(color='green',icon='ok-sign')

# for index,row in spray2011.iterrows():
#     folium.Marker([row['Latitude'], row['Longitude']], icon=icon2011,
#                   popup=str(row['Date'], 
#                   )).add_to(markerspray)
                       

# for index, row2 in spray2013.iterrows():
#     folium.Marker([row2['Latitude'], row2['Longitude']], icon=icon2013, 
#                   popup=str(row2['Date'] 
#                   )).add_to(markerspray)
    
# markerspray.save('/Users/skiparay/Desktop/Marker Spray.html')
   

    
#This Currently does not work.  I'll try and figure it out.